In [77]:
import random
from collections import defaultdict
import pandas as pd
import numpy as np

#读入数据集
ua_base = pd.read_csv('ml-100k/ua.base', sep='\t', names=['uid', 'iid', 'rate', 'timestamp'])
ua_test = pd.read_csv('ml-100k/ua.test', sep='\t', names=['uid', 'iid', 'rate', 'timestamp'])


#定义误差函数
def ERR(test):
    cnt = 0
    abs_err = 0
    squ_err = 0

    for index, row in test.iterrows():
        user_id = row['uid'] - 1
        item_id = row['iid'] - 1
        true_rating = row['rate']

        predicted_rating = R_HAT(user_id, item_id, U, V, W, user_bias, item_bias, miu)
        if predicted_rating > 5:
            predicted_rating = 5
        if predicted_rating < 1:
            predicted_rating = 1

            # 计算绝对误差/平方误差
        absolute_error = abs(predicted_rating - true_rating)
        abs_err += absolute_error

        square_error = pow(predicted_rating - true_rating, 2)
        squ_err += square_error

        cnt += 1
    # 计算平均绝对误差/平方误差
    mae = abs_err / cnt
    rmse = (squ_err / cnt) ** 0.5
    return mae, rmse


#初始化
user_num = ua_base['uid'].max()
item_num = ua_base['iid'].max()

rating_matrix = np.zeros((user_num, item_num), float)
y_ui = np.zeros((user_num, item_num), int)

#base记录转化为matrix
for index, row in ua_base.iterrows():
    user_id = row['uid']
    item_id = row['iid']
    rating = row['rate']
    rating_matrix[user_id - 1, item_id - 1] = rating
    y_ui[user_id - 1, item_id - 1] = 1

#全局平均
GlobalAverage = rating_matrix.sum() / y_ui.sum()

#计算四个参数 user_means item_means user_bias item_bias
rating_sum_row = [sum(row) for row in rating_matrix]
y_sum_row = [sum(row) for row in y_ui]

rating_sum_col = [sum(column) for column in zip(*rating_matrix)]
y_sum_col = [sum(column) for column in zip(*y_ui)]

user_means = []
for i in range(user_num):
    if y_sum_row[i] == 0:
        user_means.append(GlobalAverage)
    else:
        user_means.append(rating_sum_row[i] / y_sum_row[i])

item_means = []
for i in range(item_num):
    if y_sum_col[i] == 0:
        item_means.append(GlobalAverage)
    else:
        item_means.append(rating_sum_col[i] / y_sum_col[i])

user_bias = []
for i in range(user_num):
    if y_sum_row[i] == 0:
        user_bias.append(0)
    else:
        sum_bias = 0
        for j in range(item_num):
            sum_bias += y_ui[i][j] * (rating_matrix[i][j] - GlobalAverage)
        user_bias.append(sum_bias / y_sum_row[i])

item_bias = []
for i in range(item_num):
    if y_sum_col[i] == 0:
        item_bias.append(0)
    else:
        sum_bias = 0
        for j in range(user_num):
            sum_bias += y_ui[j][i] * (rating_matrix[j][i] - GlobalAverage)
        item_bias.append(sum_bias / y_sum_col[i])

In [78]:
ua_base_num = len(ua_base)
ua_base_num_half = int(ua_base_num / 2)
indices = list(range(ua_base_num))
random.shuffle(indices)

ua_implicit = defaultdict(list)
ua_explicit = np.zeros((ua_base_num_half, ua_base_num_half), float)

for r in range(ua_base_num_half):
    row = ua_base.iloc[indices[r]]
    user_id = row['uid'] - 1
    item_id = row['iid'] - 1
    ua_implicit[user_id].append(item_id)

In [79]:
alpha_u = alpha_v = alpha_w = beta_u = beta_v = 0.01
lr = 0.01
d = 20
epoch = 100

U = np.random.rand(user_num, d)
V = np.random.rand(item_num, d)
W = np.random.rand(item_num, d)
U = (U - 0.5) * 0.01
V = (V - 0.5) * 0.01
W = (W - 0.5) * 0.01
miu = GlobalAverage

In [80]:
def U_Tilde_u(u, W):
    I_tilde_u = ua_implicit[u]
    U_tilde_u = np.zeros(d, float)
    for i_pie in I_tilde_u:
        U_tilde_u += W[i_pie]
    U_tilde_u /= len(I_tilde_u) ** 0.5
    return U_tilde_u

In [81]:
def R_HAT(u, i, U, V, W, bu, bi, miu):
    U_tilde_u = U_Tilde_u(u, W)
    return U[u] @ V[i].T + U_tilde_u @ V[i].T + bu[u] + bi[i] + miu

In [82]:
for t in range(epoch):
    for n in range(ua_base_num_half, ua_base_num):
        print(f'\r{t + 1}:{n - ua_base_num_half + 1}', end='')
        row = ua_base.iloc[indices[n]]
        user_id = row['uid'] - 1
        item_id = row['iid'] - 1
        rating = row['rate']

        e_ui = rating - R_HAT(user_id, item_id, U, V, W, user_bias, item_bias, miu)
        #计算梯度

        delta_Uu = -e_ui * V[item_id] + alpha_u * U[user_id]
        delta_Vi = -e_ui * (U[user_id] + U_Tilde_u(user_id, W)) + alpha_v * V[item_id]
        delta_bu = -e_ui + beta_u * user_bias[user_id]
        delta_bi = -e_ui + beta_v * item_bias[item_id]
        delta_miu = -e_ui

        for i_pie in ua_implicit[user_id]:
            if len(ua_implicit[user_id]) == 0:
                continue
            delta_W = -e_ui / len(ua_implicit[user_id]) * V[item_id] + alpha_w * W[i_pie]
            #update 
            W[i_pie] -= lr * delta_W

        miu -= lr * delta_miu
        user_bias[user_id] -= lr * delta_bu
        item_bias[item_id] -= lr * delta_bi
        U[user_id] -= lr * delta_Uu
        V[item_id] -= lr * delta_Vi
    lr *= 0.9

100:45285

In [83]:
mae, rmse = ERR(ua_test)
print(f'(RMSE:{rmse:.4f},MAE:{mae:.4f})')  

(RMSE:0.9736,MAE:0.7671)
